In [ ]:
#1. Extraction
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter
from astropy.io import fits
Base= fits.open("rayonsxfiles.fits")
data=Base[1].data
Flux=data['rayonnement']
temps=data['TIME']
#2. transfert des différents rayonnements

In [9]:
import numpy as np
import os
from astropy.io import fits
from scipy.io.wavfile import write
from IPython.display import Audio

# -----------------------------
# PARAMÈTRES
# -----------------------------
FS = 44100
DURATION = 5
OUTPUT_DIR = "sons"
RAYONNEMENTS = {
    "Radio": (50, 150),
    "Micro-ondes": (150, 300),
    "Infrarouge": (300, 600),
    "Visible": (600, 1200),
    "Ultraviolet": (1200, 2500),
    "Rayons X": (2500, 6000),
    "Gamma": (6000, 12000)
}

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------------
# CHARGEMENT DU FITS
# -----------------------------
fits_file = "jw04043-o001_t001_nircam_clear-f444w_i2d.fits"  # chemin sur Colab
with fits.open(fits_file) as hdul:
    data = hdul[1].data


data = np.nan_to_num(data)
data_1d = data.flatten()
data_1d = (data_1d - np.min(data_1d)) / (np.max(data_1d) - np.min(data_1d))  # normalisation

# -----------------------------
# CHOIX DU RAYONNEMENT
# -----------------------------
rayonnement = "Infrarouge"  # JWST → infrarouge
f_min, f_max = RAYONNEMENTS[rayonnement]

# -----------------------------
# SONIFICATION
# -----------------------------
t = np.linspace(0, DURATION, int(FS * DURATION))
data_interp = np.interp(np.linspace(0, len(data_1d), len(t)), np.arange(len(data_1d)), data_1d)
frequencies = f_min + data_interp * (f_max - f_min)
signal = np.sin(2 * np.pi * frequencies * t)
signal /= np.max(np.abs(signal))

# -----------------------------
# SAUVEGARDE DU SON
# -----------------------------
output_file = f"{OUTPUT_DIR}/{rayonnement.replace(' ', '_')}.wav"
write(output_file, FS, signal.astype(np.float32))
print(f"🎧 Son généré : {output_file}")

# -----------------------------
# ÉCOUTE DIRECTE
# -----------------------------
Audio(signal, rate=FS)

🎧 Son généré : sons/Infrarouge.wav


In [10]:
from astropy.io import fits

fits_file = "jw04043-o001_t001_nircam_clear-f444w_i2d.fits"

with fits.open(fits_file) as hdul:
    hdul.info()

Filename: jw04043-o001_t001_nircam_clear-f444w_i2d.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     274   ()      
  1  SCI           1 ImageHDU        75   (5246, 2852)   float32   
  2  ERR           1 ImageHDU        10   (5246, 2852)   float32   
  3  CON           1 ImageHDU        10   (5246, 2852, 1)   int32   
  4  WHT           1 ImageHDU         9   (5246, 2852)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (5246, 2852)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (5246, 2852)   float32   
  7  VAR_FLAT      1 ImageHDU         9   (5246, 2852)   float32   
  8  HDRTAB        1 BinTableHDU    532   12R x 261C   [23A, 5A, 3A, 48A, 7A, 13A, 3A, 5A, 6A, 7A, 10A, 4A, L, D, D, 32A, 50A, 69A, 14A, 2A, 8A, 10A, 12A, 23A, 23A, 26A, 11A, 5A, 3A, 3A, 2A, 1A, 2A, 1A, L, 39A, 8A, 2A, 26A, 20A, 27A, 10A, K, L, L, L, L, 10A, 1A, 5A, D, D, D, D, D, D, D, D, 6A, 8A, 1A, 4A, 5A, 5A, L, 4A, K, 5A, 9A, D, D, D, D, D, D, D,

In [11]:
!pip install pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [16]:
import numpy as np
import os
from astropy.io import fits
from scipy.io.wavfile import write
from pydub import AudioSegment
from IPython.display import Audio, display

# -----------------------------
# PARAMÈTRES
# -----------------------------
FS = 44100
DURATION = 5
OUTPUT_DIR = "sons"
RAYONNEMENTS = {
    "Radio": (50, 150),
    "Micro-ondes": (150, 300),
    "Infrarouge": (300, 600),
    "Visible": (600, 1200),
    "Ultraviolet": (1200, 2500),
    "Rayons X": (2500, 6000),
    "Gamma": (6000, 12000)
}

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------------
# CHARGEMENT DU FITS
# -----------------------------
fits_file = "jw04043-o001_t001_nircam_clear-f444w_i2d.fits"

with fits.open(fits_file) as hdul:
    data = hdul[1].data  # HDU SCI

data = np.nan_to_num(data)
data_1d = data.flatten()
data_1d = (data_1d - np.min(data_1d)) / (np.max(data_1d) - np.min(data_1d))

# -----------------------------
# DEMANDE DU RAYONNEMENT
# -----------------------------
print("Types de rayonnement disponibles :")
for r in RAYONNEMENTS.keys():
    print(f"- {r}")

rayonnement = input("Quel type de rayonnement contient ce fichier ? ").strip()
if rayonnement not in RAYONNEMENTS:
    raise ValueError("Type de rayonnement non reconnu !")

f_min, f_max = RAYONNEMENTS[rayonnement]

# -----------------------------
# SONIFICATION
# -----------------------------
t = np.linspace(0, DURATION, int(FS * DURATION))
data_interp = np.interp(
    np.linspace(0, len(data_1d), len(t)),
    np.arange(len(data_1d)),
    data_1d
)

frequencies = f_min + data_interp * (f_max - f_min)

# Affiche un aperçu des fréquences
print(f"Aperçu des fréquences : min = {frequencies.min():.2f} Hz, max = {frequencies.max():.2f} Hz")

signal = np.sin(2 * np.pi * frequencies * t)
signal /= np.max(np.abs(signal))  # normalisation

# -----------------------------
# SAUVEGARDE WAV TEMPORAIRE
# -----------------------------
wav_file = f"{OUTPUT_DIR}/{rayonnement.replace(' ', '_')}.wav"
write(wav_file, FS, (signal * 32767).astype(np.int16))  # WAV 16-bit

# -----------------------------
# CONVERSION EN MP3
# -----------------------------
mp3_file = f"{OUTPUT_DIR}/{rayonnement.replace(' ', '_')}.mp3"
audio_segment = AudioSegment.from_wav(wav_file)
audio_segment.export(mp3_file, format="mp3")
print(f"🎧 Fichier audio MP3 généré : {mp3_file}")

# -----------------------------
# ÉCOUTE DIRECTE
# -----------------------------
display(Audio(signal, rate=FS))

Types de rayonnement disponibles :
- Radio
- Micro-ondes
- Infrarouge
- Visible
- Ultraviolet
- Rayons X
- Gamma
Quel type de rayonnement contient ce fichier ? rayon


ValueError: Type de rayonnement non reconnu !

In [36]:
import numpy as np
import os
from astropy.io import fits
from scipy.io.wavfile import write
from pydub import AudioSegment
from IPython.display import Audio, display

# -----------------------------
# PARAMÈTRES
# -----------------------------
FS = 44100
DURATION = 10      # durée totale audio
OUTPUT_DIR = "sons"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Ranges audibles, on pourra les ajuster
RAYONNEMENTS = {
    "Radio": (50, 150),
    "Micro-ondes": (150, 300),
    "Infrarouge": (300, 600),
    "Visible": (600, 1200),
    "Ultraviolet": (1200, 2500),
    "Rayons X": (2500, 6000),
    "Gamma": (6000, 12000)
}

# -----------------------------
# CHARGEMENT DU FITS TIMESERIES
# -----------------------------
fits_file = "tess2018349182500-s0006-0000000269273552-0126-s_lc.fits"  # remplace ici par ton fichier

with fits.open(fits_file) as hdul:
    # Affiche la structure pour valider
    hdul.info()

    # La table devrait être dans la première extension
    table = hdul[1].data

# extraire temps et flux
time = table["TIME"]
flux = table["PDCSAP_FLUX"]  # flux corrigé
flux = np.nan_to_num(flux)

# normalisation de flux entre 0 et 1
flux_norm = (flux - np.min(flux)) / (np.max(flux) - np.min(flux))

# -----------------------------
# CHOIX DU RAYONNEMENT
# -----------------------------
print("Rayonnements disponibles :")
for key in RAYONNEMENTS.keys():
    print("-", key)

ray_type = input("Quel type de rayonnement ? ").strip()
if ray_type not in RAYONNEMENTS:
    raise ValueError("Rayonnement non reconnu")

f_min, f_max = RAYONNEMENTS[ray_type]

# -----------------------------
# SONIFICATION TEMPS → FREQUENCE
# -----------------------------
t = np.linspace(0, DURATION, int(FS * DURATION))

# interpole la série normale
flux_interp = np.interp(
    np.linspace(0, len(flux_norm), len(t)),
    np.arange(len(flux_norm)),
    flux_norm
)

# conversion vers une bande audible
frequencies = f_min + flux_interp * (f_max - f_min)

print(f"Aperçu des fréquences générées :  min = {frequencies.min():.2f} Hz,  max = {frequencies.max():.2f} Hz")

signal = np.sin(2 * np.pi * frequencies * t)
signal /= np.max(np.abs(signal))

# -----------------------------
# SAUVEGARDE WAV
# -----------------------------
wav_file = f"{OUTPUT_DIR}/{ray_type.replace(' ', '_')}_toi1231.wav"
write(wav_file, FS, (signal * 32767).astype(np.int16))
print("✔️ Fichier WAV généré:", wav_file)

# -----------------------------
# CONVERSION EN MP3
# -----------------------------
mp3_file = f"{OUTPUT_DIR}/{ray_type.replace(' ', '_')}_explosion.mp3"
audio_segment = AudioSegment.from_wav(wav_file)
audio_segment.export(mp3_file, format="mp3")
print("🎧 MP3 exporté :", mp3_file)

# -----------------------------
# ÉCOUTE DIRECTE
# -----------------------------
display(Audio(signal, rate=FS))


Filename: tess2018349182500-s0006-0000000269273552-0126-s_lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  LIGHTCURVE    1 BinTableHDU    167   15678R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        49   (25, 25)   int32   
Rayonnements disponibles :
- Radio
- Micro-ondes
- Infrarouge
- Visible
- Ultraviolet
- Rayons X
- Gamma
Quel type de rayonnement ? Radio
Aperçu des fréquences générées :  min = 50.00 Hz,  max = 149.97 Hz
✔️ Fichier WAV généré: sons/Radio_toi1231.wav
🎧 MP3 exporté : sons/Radio_explosion.mp3
